In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import math
import os
import pprint
import logging
import json
import random

import datasets
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import numpy as np
import torch
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import AdamW, get_scheduler, set_seed

from transformers.file_utils import is_offline_mode
from transformers.utils.versions import require_version

from args import parse_args
from data_loader import raw_data_loader, data_processor
from model_loader import model_loader
from rouge_s import py_rouge_scores
from utils import label_smoothed_nll_loss, postprocess_text
from contrastive_loss import margin_ranking_loss, cosine_embedding_loss

[nltk_data] Downloading package punkt to /home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [4]:
from transformers import SchedulerType
import argparse

In [5]:
arg_parser = argparse.ArgumentParser(description="bart")
arg_parser.add_argument("--topic_prompt_input", dest="topic_prompt_input", type=bool,
                        default=False, help="Use topic prompt or not")
arg_parser.add_argument("--length_prompt_input", dest="length_prompt_input", type=bool,
                        default=False, help="Use length prompt or not")
arg_parser.add_argument("--predict_summary", dest="predict_summary", type=bool,
                        default=False, help="Use predict summary or not")
arg_parser.add_argument("--output_dir", dest="output_dir",
                        type=str, default="./output/1", help="default")
arg_parser.add_argument("--train_file", dest="train_file", type=str,
                        default=None, help="A json file containing the training data.")
arg_parser.add_argument("--validation_file", dest="validation_file", type=str,
                        default=None, help="A json file containing the validation data.")
arg_parser.add_argument("--test_file", dest="test_file", type=str,
                        default=None, help="A json file containing the test data.")
arg_parser.add_argument("--ignore_pad_token_for_loss", dest="ignore_pad_token_for_loss", type=bool, default=True,
                        help="Whether to ignore the tokens corresponding to padded labels in the loss computation or not.",)
arg_parser.add_argument("--text_column", dest="text_column", type=str, default="dialogue",
                        help="The name of the column in the datasets containing the full texts (for summarization).")
arg_parser.add_argument("--summary_column", dest="summary_column", type=str, default="summary",
                        help="The name of the column in the datasets containing the summaries (for summarization).")
arg_parser.add_argument("--model_name_or_path", dest="model_name_or_path", type=str, default="facebook/bart-large",
                        help="Path to pretrained model or model identifier from huggingface.co/models.")
arg_parser.add_argument("--model_type", dest="model_type", type=str, default="bart",
                        help="Model type to use if training from scratch.")
arg_parser.add_argument("--max_source_length", dest="max_source_length", 
                        type=int, default=1024, help="default")
arg_parser.add_argument("--preprocessing_num_workers", type=int, default=None,
                        help="The number of processes to use for the preprocessing.")
arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=bool,
                        default=None, help="Overwrite the cached training and evaluation sets")
arg_parser.add_argument("--min_target_length", dest="min_target_length", type=int,
                        default=1, help="The minimal total sequence length for target text")
arg_parser.add_argument("--max_target_length", dest="max_target_length", type=int, default=128, help="The maximum total sequence length for target text"
                        "after tokenization. Sequences longer than this will be truncated, sequences shorter will be padded. during ``evaluate`` and"
                        "``predict``.")
arg_parser.add_argument("--num_beams", dest="num_beams", type=int, default=4, help="Number of beams to use for evaluation. This argument will be "
                        "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--learning_rate", dest="learning_rate", type=float, default=5e-5,
                        help="Initial learning rate (after the potential warmup period) to use.")
arg_parser.add_argument("--pad_to_max_length", action="store_true",
                        help="If passed, pad all samples to `max_length`. Otherwise, dynamic padding is used.",)
arg_parser.add_argument("--weight_decay", dest="weight_decay",
                        type=float, default=1e-3, help="Weight decay to use.")
arg_parser.add_argument("--label_smoothing", dest="label_smoothing",
                        type=float, default=0.1, help="hyperparameter for label smoothing.")
arg_parser.add_argument("--length_penalty", dest="length_penalty", type=float,
                        default=1.0, help="large - longer sequence, small - shorter sequence")
arg_parser.add_argument("--num_train_epochs", dest="num_train_epochs",
                        type=int, default=15, help="Total number of training epochs to perform.")
arg_parser.add_argument("--per_device_train_batch_size", dest="per_device_train_batch_size",
                        type=int, default=8, help="Batch size (per device) for the training dataloader.")
arg_parser.add_argument("--gradient_accumulation_steps", dest="gradient_accumulation_steps", type=int,
                        default=64, help="Number of updates steps to accumulate before performing a backward/update pass.")
arg_parser.add_argument("--per_device_eval_batch_size", dest="per_device_eval_batch_size",
                        type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--per_device_test_batch_size", dest="per_device_test_batch_size",
                        type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--num_warmup_steps", dest="num_warmup_steps", type=int,
                        default=0, help="Number of steps for the warmup in the lr scheduler.")
arg_parser.add_argument("--cache_dir", dest="cache_dir",
                        type=str, default="./output/cache", help="default")
arg_parser.add_argument("--seed", dest="seed",
                        type=int, default=12345, help="default")
arg_parser.add_argument("--config_name", type=str, default=None,
                        help="Pretrained config name or path if not the same as model_name")
arg_parser.add_argument("--tokenizer_name", type=str, default=None,
                        help="Pretrained tokenizer name or path if not the same as model_name")
arg_parser.add_argument("--use_slow_tokenizer", dest="use_slow_tokenizer", action="store_true",
                        help="If passed, will use a slow tokenizer (not backed by the HuggingFaceTokenizers library).")
arg_parser.add_argument("--max_train_steps", type=int, default=None,
                        help="Total number of training steps to perform. If provided, overrides num_train_epochs.")
arg_parser.add_argument("--lr_scheduler_type", type=SchedulerType, default="linear", help="The scheduler type to use.",
                        choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"])
arg_parser.add_argument("--embedding_lr", type=float, default=5e-5,
                        help="Initial learning rate for embedding layers.")
arg_parser.add_argument("--len_start", type=int,
                        default=1, help="start length.")
arg_parser.add_argument("--len_end", type=int,
                        default=100, help="end length.")
arg_parser.add_argument("--contrastive_loss", dest="contrastive_loss", type=bool,
                        default=False, help="Use contrastive loss or not")
arg_parser.add_argument("--tagging", dest="tagging", type=str, default="no",
                        choices=('no', 'word', 'prompt'), help="Use tagging (<tp>, </tp>) in word, sentence, or not")
arg_parser.add_argument("--synonym_replacement", dest="synonym_replacement", type=bool,
                        default=False, help="Synonym replacement or not")
arg_parser.add_argument("--random_topic", dest="random_topic", type=bool,
                        default=False, help="Random topic or not")
arg_parser.add_argument("--alpha", dest="alpha", type=float,
                        default=0.5, help="ration of computation loss in encoder")
arg_parser.add_argument("--margin", dest="margin", type=float,
                        default=0, help="margin of computation loss")
arg_parser.add_argument("--debug", action='store_true',
                        default=False, help="Use the debug mode or not")

_StoreTrueAction(option_strings=['--debug'], dest='debug', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Use the debug mode or not', metavar=None)

In [6]:
args = arg_parser.parse_args('')

In [7]:
args.train_file = "./data/dialogtest/dialogsum.train.jsonl"
args.validation_file = "./data/dialogtest/dialogsum.dev.jsonl"
args.test_file = "./data/dialogtest/dialogsum.test.jsonl"
args.text_column = "prompt"
args.summary_column = "summary"
args.model_name_or_path = "facebook/bart-large"
args.model_type = "bart"
args.max_source_length = 1024
args.min_target_length = 1
args.max_target_length = 128
args.num_beams = 4
args.learning_rate = 5e-5
args.weight_decay = 1e-3
args.label_smoothing = 0.1
args.length_penalty = 1.0
args.num_train_epochs = 1
args.per_device_train_batch_size = 2
args.gradient_accumulation_steps = 64
args.per_device_eval_batch_size = 8
args.per_device_test_batch_size = 8
args.num_warmup_steps = 0
args.cache_dir = "./output/cache"
args.overwrite_cache = True
args.seed = 12345
args.output_dir = "./output/bart-test"

In [8]:
# If passed along, set the training seed now.
logging.info("*** Set Seed ***")
logger.info(args.seed)
set_seed(args.seed)
random.seed(args.seed)
os.environ['PYTHONHASHSEED'] = str(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

11/18/2023 09:05:11 - INFO - root - *** Set Seed ***
11/18/2023 09:05:11 - INFO - __main__ - 12345


In [9]:
args.topic_prompt_input = True
# args.topic_prompt_input = False
# args.length_prompt_input = True
args.length_prompt_input = False
args.contrastive_loss = True
# args.contrastive_loss = False
# args.tagging = "no"
# args.tagging = "word"
# args.tagging = "prompt"
args.synonym_replacement = True
# args.synonym_replacement = False
args.random_topic = True
# args.random_topic = False

In [10]:
raw_datasets = raw_data_loader(args)

In [11]:
raw_datasets['train'].features

{'id': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'topic': Value(dtype='string', id=None),
 'synonym_prompt': Value(dtype='string', id=None),
 'synonym_topic': Value(dtype='string', id=None),
 'random_prompt': Value(dtype='string', id=None),
 'random_topic': Value(dtype='string', id=None)}

In [12]:
num = 4
for feature in raw_datasets['train'].features:
    # if feature == "topic" or feature == "synonym_topic" or feature == "random_topic":
    #     print(raw_datasets['train'][feature][num])
    print(raw_datasets['train'][feature][num])
    print("-"*20)

train_4
--------------------
Topic of Summary: dance. Dialogue: #Person1#: Watsup, ladies! Y'll looking'fine tonight. May I have this dance?
#Person2#: He's cute! He looks like Tiger Woods! But, I can't dance. . .
#Person1#: It's all good. I'll show you all the right moves. My name's Malik.
#Person2#: Nice to meet you. I'm Wen, and this is Nikki.
#Person1#: How you feeling', vista? Mind if I take your friend'round the dance floor?
#Person2#: She doesn't mind if you don't mind getting your feet stepped on.
#Person1#: Right. Cool! Let's go!
--------------------
Malik invites Nikki to dance. Nikki agrees if Malik doesn't mind getting his feet stepped on.
--------------------
dance
--------------------
Topic of Summary: dancing. Dialogue: #Person1#: Watsup, ladies! Y'll looking'fine tonight. May I have this dance?
#Person2#: He's cute! He looks like Tiger Woods! But, I can't dance. . .
#Person1#: It's all good. I'll show you all the right moves. My name's Malik.
#Person2#: Nice to meet you

In [ ]:
accelerator = Accelerator(mixed_precision="fp16")
logger.info(accelerator.state)

In [ ]:
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
accelerator.is_local_main_process
datasets.utils.logging.set_verbosity_warning()
transformers.utils.logging.set_verbosity_info()
set_seed(args.seed)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
accelerator.is_main_process
os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [ ]:
config, tokenizer, model = model_loader(accelerator, logger, args)

In [ ]:
model.vocab_size

In [ ]:
print(tokenizer.additional_special_tokens)

In [ ]:
dataloader, processed_dataset = data_processor(logger, args, accelerator, raw_datasets, tokenizer, model)
train_dataloader, eval_dataloader, test_dataloader = dataloader
train_dataset, _, _ = processed_dataset

In [ ]:
train_dataset

In [ ]:
for step, batch in enumerate(train_dataloader):
    for ind, batch_keys in enumerate(batch.keys()):
        print(batch[batch_keys].shape)
        for indx in range(batch[batch_keys].shape[0]):
            print("index: ", indx)
            if batch_keys == 'labels':
                batch[batch_keys][indx] = torch.where(batch[batch_keys][indx] != -100, batch[batch_keys][indx], tokenizer.pad_token_id)
                print(tokenizer.decode((batch[batch_keys][indx]), skip_special_tokens=True))
            else:
                print(tokenizer.decode((batch[batch_keys][indx])))
                # print(tokenizer.decode((batch[batch_keys][indx]), skip_special_tokens=True))
    if step == 1:
        break

In [ ]:
for step, batch in enumerate(test_dataloader):
    print(batch.keys())
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['labels'].shape)
    print(batch['decoder_input_ids'].shape)
    break

In [ ]:
for step, batch in enumerate(eval_dataloader):
    print(batch.keys())
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['labels'].shape)
    print(batch['decoder_input_ids'].shape)
    break

In [ ]:
# = = = Training Preparation = = =
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]

no_decay_emb_matrix = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay_emb_matrix)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

# optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

In [ ]:
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

In [ ]:
# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ", disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses  = []
best_r2_f1  = None
best_epoch  = 0

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Train =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
for epoch in range(args.num_train_epochs):
    # train
    model.train()
    for step, batch in enumerate(train_dataloader):
        if args.label_smoothing == 0:
            outputs = model(**batch)
            loss = outputs.loss
        else:
            outputs = model(**batch, output_hidden_states=True)
            output_logits = outputs.logits

            output_probs = torch.nn.functional.log_softmax(
                output_logits, dim=-1)

            if args.contrastive_loss:
                max_encoder_token = model.config.max_position_embeddings

                embeddings = outputs.encoder_last_hidden_state[:args.per_device_train_batch_size,:,:max_encoder_token]
                # print(embeddings.shape)
                embeddings = embeddings.reshape(-1, max_encoder_token)
                
                minus_one = -1 * torch.ones(embeddings.size(dim=0)).to(device)

                if args.synonym_replacement and not args.random_topic:
                    print("synonym_replacement")
                    synonym_embeddings = outputs.encoder_last_hidden_state[args.per_device_train_batch_size:args.per_device_train_batch_size*2,:,:max_encoder_token]
                    synonym_embeddings = synonym_embeddings.reshape(-1, max_encoder_token)
                    loss_cs = cosine_embedding_loss(embeddings, synonym_embeddings, minus_one, args.margin)

                elif not args.synonym_replacement and args.random_topic:
                    print("random_topic")
                    random_embeddings = outputs.encoder_last_hidden_state[args.per_device_train_batch_size:args.per_device_train_batch_size*2,:,:max_encoder_token]
                    random_embeddings = random_embeddings.reshape(-1, max_encoder_token)
                    loss_cs = cosine_embedding_loss(embeddings, random_embeddings, minus_one, args.margin)

                elif args.synonym_replacement and args.random_topic:
                    print("all")
                    synonym_embeddings = outputs.encoder_last_hidden_state[args.per_device_train_batch_size:args.per_device_train_batch_size*2,:,:max_encoder_token]
                    random_embeddings = outputs.encoder_last_hidden_state[args.per_device_train_batch_size*2:,:,:max_encoder_token]
                    synonym_embeddings = synonym_embeddings.reshape(-1, max_encoder_token)
                    random_embeddings = random_embeddings.reshape(-1, max_encoder_token)
                    loss_cs_synonym = cosine_embedding_loss(embeddings, synonym_embeddings, minus_one, args.margin)
                    loss_cs_random = cosine_embedding_loss(embeddings, random_embeddings, minus_one, args.margin)
                    loss_cs = (loss_cs_synonym + loss_cs_random) / 2

                # negative log-likelihood
                output_probs = output_probs[:args.per_device_train_batch_size,:,:]
                # print("output_probs:" ,output_probs.shape)
                output_probs = output_probs.view(-1, model.config.vocab_size)
                # print("output_probs:" ,output_probs.shape)
                gt_logits = batch['labels'][:args.per_device_train_batch_size,:]
                # print("gt_logits:" ,gt_logits.shape)
                # gt_logits_0 = torch.where(gt_logits[0] != -100, gt_logits[0], tokenizer.pad_token_id)
                # gt_logits_1 = torch.where(gt_logits[1] != -100, gt_logits[1], tokenizer.pad_token_id)
                # print(tokenizer.decode(gt_logits_0))
                # print(tokenizer.decode(gt_logits_1))
                gt_logits = gt_logits.view(-1)
                # print("gt_logits:" ,gt_logits.shape)
                loss_nll, nll = label_smoothed_nll_loss(output_probs, gt_logits, args.label_smoothing, ignore_index=tokenizer.pad_token_id)

                # joint loss
                loss = loss_nll + (args.alpha * loss_cs)
                break
            else:
                # print("no contrastive")
                # print("output_probs:" ,output_probs.shape)
                output_probs = output_probs.view(-1,model.config.vocab_size)
                # print("output_probs:" ,output_probs.shape)
                gt_logits = batch['labels']
                # print("gt_logits:" ,gt_logits.shape)
                gt_logits = gt_logits.view(-1)
                # print("gt_logits:" ,gt_logits.shape)
                loss_nll, nll = label_smoothed_nll_loss(output_probs, gt_logits, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
                loss = loss_nll
                # break

        acc_losses.append(loss.item())
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)

        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            progress_bar.set_postfix(lr=lr_scheduler.get_last_lr()[0], loss=np.mean(acc_losses[-50:]))
            completed_steps += 1

        if completed_steps >= args.max_train_steps:
            break

In [ ]:
embeddings = outputs.encoder_last_hidden_state[:args.per_device_train_batch_size,:,:max_encoder_token]

In [ ]:
embeddings[0] == outputs.encoder_last_hidden_state[0]

In [ ]:
embeddings[1] == outputs.encoder_last_hidden_state[1]

In [ ]:
synonym_embeddings = outputs.encoder_last_hidden_state[args.per_device_train_batch_size:args.per_device_train_batch_size*2,:,:max_encoder_token]
random_embeddings = outputs.encoder_last_hidden_state[args.per_device_train_batch_size*2:,:,:max_encoder_token]

In [ ]:
outputs.encoder_last_hidden_state[2] == synonym_embeddings[0]

In [ ]:
outputs.encoder_last_hidden_state[3] == synonym_embeddings[1]

In [ ]:
outputs.encoder_last_hidden_state[4] == random_embeddings[0]

In [ ]:
outputs.encoder_last_hidden_state[5] == random_embeddings[1]

In [ ]:
# with open(args.output_dir+'/gen_samples/'+'loss.json', 'w') as f:
with open('./'+'loss.json', 'w') as f:
    json_string = json.dumps(acc_losses)
    f.write(json_string)

In [ ]:
# cs_loss = nn.CosineEmbeddingLoss(margin=0.5)
cs_loss = torch.nn.CosineEmbeddingLoss()

In [ ]:
contrastive = -1 * torch.ones(embeddings.size(dim=0)).to(device)

In [ ]:
loss_cosine_embedding = cs_loss(embeddings, positive_embeddings, contrastive)

In [ ]:
loss_cosine_embedding

In [ ]:
cos = torch.nn.CosineSimilarity(dim=-1)

In [ ]:
s = cos(embeddings, positive_embeddings)

In [ ]:
s.mean()

In [ ]:
# a = torch.mean(torch.relu(0.5 - s))
a = torch.mean(torch.max(torch.zeros(s.shape[0]).to(device), (0.5 - s)))

In [ ]:
a

In [ ]:
contrastive = -1 * torch.ones(embeddings.size(dim=0)).to(device)

In [ ]:
loss_cosine_embedding = cs_loss(embeddings, negative_embeddings, contrastive)

In [ ]:
loss_cosine_embedding

In [ ]:
n = cos(embeddings, negative_embeddings)

In [ ]:
n.mean()

In [ ]:
# b = torch.mean(torch.relu(0.5 - n))
b = torch.mean(torch.max(torch.zeros(n.shape[0]).to(device), (0.5 - n)))

In [ ]:
b

In [ ]:
s.shape

In [ ]:
# Calculate distances
pos_distance = torch.norm(embeddings - positive_embeddings, p=2, dim=1)
neg_distance = torch.norm(embeddings - negative_embeddings, p=2, dim=1)
# Calculate loss
loss = torch.mean(torch.relu(pos_distance - neg_distance + 0))

In [ ]:
pos_distance.mean()

In [ ]:
neg_distance.mean()

In [ ]:
loss

In [ ]:
outputs.encoder_last_hidden_state[:2,:,:].shape

In [ ]:
outputs.encoder_last_hidden_state[2:,:,:].shape

In [ ]:
output_probs.shape

In [ ]:
int(output_probs.shape[0] / 2)

In [ ]:
output_probs_pos = output_probs[:2,:,:]
output_probs_pos = output_probs_pos.view(-1,
                                 model.config.vocab_size)
output_probs_neg = output_probs[2:,:,:]
output_probs_neg = output_probs_neg.view(-1,
                                 model.config.vocab_size)
gt_logits = batch['labels'][:2]
gt_logits = gt_logits.view(-1)

In [ ]:
output_probs_pos.shape

In [ ]:
output_probs_neg.shape

In [ ]:
gt_logits.shape

In [ ]:
probs_pos = output_probs_pos[~gt_logits.eq(-100)]
probs_neg = output_probs_neg[~gt_logits.eq(-100)]

In [ ]:
gt = gt_logits[~gt_logits.eq(-100)]

In [ ]:
if gt.dim() == probs_pos.dim() - 1:
    gt = gt.unsqueeze(-1)

In [ ]:
nll_pos = -probs_pos.gather(dim=-1, index=gt)
nll_neg = -probs_neg.gather(dim=-1, index=gt)

In [ ]:
nll_sq_pos = nll_pos.squeeze(-1)
nll_sq_neg = nll_neg.squeeze(-1)

In [ ]:
target_one = torch.ones(nll_sq_pos.shape[0])

In [ ]:
import torch.nn as nn
loss = nn.MarginRankingLoss()
input1 = nll_sq_pos
input2 = nll_sq_neg
target = target_one.to("cuda:0")
output = loss(input1, input2, target)
# output.backward()

In [ ]:
output

In [ ]:
len(outputs.encoder_last_hidden_state)

In [ ]:
outputs.encoder_last_hidden_state

In [ ]:
outputs.encoder_last_hidden_state.size(dim=1)

In [ ]:
print(len(outputs.logits))
print(outputs.logits[0].size())

In [ ]:
cosine_loss = torch.nn.CosineEmbeddingLoss()

In [ ]:
loss_cs = cosine_loss(outputs.encoder_last_hidden_state[0], outputs.encoder_last_hidden_state[1], torch.ones(outputs.encoder_last_hidden_state.size(dim=1)).to(torch.device('cuda')))

In [ ]:
loss_cs

In [ ]:
# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Test =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
# load best model
logger.info("Loading Best Result is at epoch {} for Testing".format(best_epoch))

unwrapped_model = accelerator.unwrap_model(model)
config          = config.from_pretrained(args.output_dir+'/best')
tokenizer       = tokenizer.from_pretrained(args.output_dir+'/best', config=config)
unwrapped_model = unwrapped_model.from_pretrained(args.output_dir+'/best', config=config)
model           = accelerator.prepare(unwrapped_model)

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

# start Test
logger.info("Collecting Testing Result...")
model.eval()

test_predict     = []
test_groundtruth = []
for step, batch in enumerate(tqdm(test_dataloader, leave=False)):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )

        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = batch["labels"]

        if not args.pad_to_max_length:
            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

        generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
        labels = accelerator.gather(labels).cpu().numpy()

        if args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        decoded_preds  = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        decoded_preds  = [' '.join(sent.split('\n')) for sent in decoded_preds]
        decoded_labels = [' '.join(sent.split('\n')) for sent in decoded_labels]

        test_predict.extend(decoded_preds)
        test_groundtruth.extend(decoded_labels)

print(raw_datasets['test']['dialogue'][0])

if args.len_output == 'real':
    new_test_predict = []
    for sample in test_predict:
        try:
            gen_sum = sample.split('Summary: ')[2]
            new_test_predict.append(gen_sum)
        except:
            new_test_predict.append(sample)
    test_predict = new_test_predict

logger.info("")
logger.info("ROUGE score on test set")
test_scores = py_rouge_scores(test_predict, test_groundtruth)
logger.info("")


# Save generated summaries
if args.len_input == 'predict':
    os.makedirs(args.output_dir+'/predict_gen_samples', exist_ok=True)
else:
    os.makedirs(args.output_dir+'/gen_samples', exist_ok=True)

for i in range(len(test_predict)):
    test_id        = raw_datasets['test']['id'][i]
    test_dialogue  = raw_datasets['test']['dialogue'][i]
    test_summary   = raw_datasets['test']['summary'][i]
    test_predict_s = test_predict[i]

    if args.len_input == 'predict':
        with open(args.output_dir+'/predict_gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)
    else:
        with open(args.output_dir+'/gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)